In [37]:
import pandas as pd
import numpy as np
import networkx as nx
import pickle
import json
import requests
import tweepy
import os

key = 'B81Sj4ifYRstLZGMyMDM0roG3'
secret = 'c1Rg4vdyl2mjMEC7WKDsVLTz2IlqTHy1yezn71Y9cw3LgZaxAW'
token = 'AAAAAAAAAAAAAAAAAAAAAD6eWQEAAAAABtT80i7lkSemhYsuOIjy4O4xLQs%3DkfsByhRoXQ9odyT51sECR1vXxOLKxdtOON2Ij4OODdAt8jTcGY'

# key='CSh1kvK1gpxzyhl8GYuMa0RIv'
# secret='yfgNtKNVeqdjqktu9lR4eK0oEt68KjXhVV8jJOJgvqrvpFbL9N'
# token='AAAAAAAAAAAAAAAAAAAAAOCeVAEAAAAABD8nomZEvbNVV3p7dFOVkhm5Kv8%3DritcqLa43rGxvmEUXtuSVp6fwDKsONk6hbgPVGyjmqI72ueB9W'

api = tweepy.Client(bearer_token=token, consumer_key=key, consumer_secret=secret, 
              return_type=requests.Response,
              wait_on_rate_limit=True)

In [254]:
def extract_uid(full_path):
    centerNode = file.replace('../','').replace('.pkl','').replace('edgelist_2_deg_','')\
    .replace('edgelist_mentions_','').replace('/','').replace('human','')\
    .replace('bot','').replace('mentions','').replace('edgelist_2_edges_from_start_', '')
    
    return int(centerNode)

In [255]:
# Returns the followers/following of the center node in the network (because these aren't in the pickle files)
def getCenterNodeFollows(file):
    centerNode = extract_uid(file)
    r = api.get_user(id=centerNode, user_fields='public_metrics')
    user = r.json()

    followers = user['data']['public_metrics']['followers_count']
    following = user['data']['public_metrics']['following_count']
    
    return followers, following, centerNode

In [256]:
def is_valid_network(df):
    if df.shape[0] == 0:
        print(f'No Network: {full_path}')
        return False
    return True

def is_valid(file, uid):
    edgelist = pd.read_pickle(file)
    edgelist.originating_id = edgelist.originating_id.astype(str)
    edgelist.receiving_id = edgelist.receiving_id.astype(str)
    t = edgelist.originating_id.append(edgelist.receiving_id).unique()
    if str(uid) in t:
        return True
    print(f'Error - Main twitter id not in edgelist: {uid} ')
    return False
    

In [257]:
# Takes in a pickle object and returns in/out degree centralities, 
# clustering coefficient, local homophily metrics, and PageRank for account ID.
def node_calculations(file):
    full_list = pd.read_pickle(file)
    
    # Check if valid
    if not is_valid_network(full_list):
        feats = [0 for i in range(8)]
        return feats
    
    if 'edge_weight' in full_list.columns:
        weightlist = full_list['edge_weight']
    else:
        weightlist = [1]*len(full_list['originating_id'])
    
    edgelist = pd.DataFrame({
        "source": full_list['originating_id'],
        "target": full_list['receiving_id'],
        "weight": weightlist,
    })
    
    # store originating ID followers
    attr = dict()
    for i, row in full_list.iterrows():
        if not pd.isna(row['originating_following_count']):
            attr[row['originating_id']] = {'in': row['originating_follower_count'], 
                                           'out': row['originating_following_count']}
        if not pd.isna(row['receiving_following_count']):
            attr[row['receiving_id']] = {'in': row['receiving_follower_count'], 
                                         'out': row['receiving_following_count']}
            
    IDfollowers, IDfollowing, ID = getCenterNodeFollows(file)
    attr[ID] = {'in': IDfollowers, 'out': IDfollowing}
    
    # NOTE: nothing in "attr" represents the degrees of nodes in OUR network; the attributes represent
    # the degrees of nodes in the FULL network
    
    #print(attr)
            
    G = nx.from_pandas_edgelist(edgelist, create_using=nx.DiGraph(), edge_attr=True)
    nx.set_node_attributes(G, attr)
    
    nodes = list(G.nodes)
    
    
    clustercoeff = nx.clustering(G, ID, weight='weight')
    pagerank = nx.pagerank(G, weight='weight')[ID]

    
    # compute "local homophily" for ID's degree structure in two different ways
    indeg = len(list(G.predecessors(ID)))
    outdeg = len(list(G.successors(ID)))
    
    outdeg_Assort1 = 0
    outdeg_Assort2 = 0
    indeg_Assort1 = 0
    indeg_Assort2 = 0
    
    if outdeg != 0:
        for node in G.neighbors(ID):
            nodeOutdeg = attr[node]['out']
            outdeg_Assort1 += nodeOutdeg
            if nodeOutdeg >= IDfollowing/2 and nodeOutdeg <= IDfollowing*2:
                outdeg_Assort2 += 1
                
        outdeg_Assort1 = outdeg_Assort1 / (outdeg*IDfollowing)
        outdeg_Assort2 = outdeg_Assort2 / outdeg

    if indeg != 0: 
        for node in G.predecessors(ID):    
            nodeIndeg = attr[node]['in']
            indeg_Assort1 += nodeIndeg
            if nodeIndeg >= IDfollowers/2 and nodeIndeg <= IDfollowers*2:
                indeg_Assort2 += 1
    
        indeg_Assort1 = indeg_Assort1 / (indeg*IDfollowers)
        indeg_Assort2 = indeg_Assort2 / indeg
    
    
    return (IDfollowers, IDfollowing, clustercoeff, pagerank, indeg_Assort1, outdeg_Assort1, \
            indeg_Assort2, outdeg_Assort2)

In [260]:
def extract_feats(full_path, df, invalid):
    # Get twitter id
    uid = extract_uid(full_path)

    # Calc metrics
    edge = pd.read_pickle(full_path)
    if not is_valid_network(edge):
        feats = [0 for i in range(8)]
    elif not is_valid(full_path, uid):
        return df, invalid
    else:
        feats = list(node_calculations(full_path))

    # Create row for df
    row = [uid]
    row.extend(feats)
    row.append(label)
    rowdf = pd.DataFrame(row).T
    rowdf.columns = cols
    df = pd.concat([df, rowdf])
    
    return df, invalid

In [261]:
# Create pd dataframe for mentions
cols = ['twitter_id', 'indeg', 'outdeg', 'clustercoeff', 'pagerank', 'indeg_A1', 
        'outdeg_A1', 'indeg_A2', 'outdeg_A2', 'label']
df = pd.DataFrame(columns=cols)

invalid = []
dirr = '../mentions/'
for label in ['bot', 'human']:
    path = dirr + label + '/'
    for file in os.listdir(dirr + label + '/'):
        if os.path.isdir(file):
            continue
        full_path = path + file
        
        # Get twitter id
        uid = extract_uid(full_path)
        
        # Another check if valid
        if not is_valid(full_path, uid):
            invalid.append(uid)
            continue
        
        # Calc metrics; create row for df
        feats = list(node_calculations(full_path))
        row = [uid]
        row.extend(feats)
        row.append(label)
        rowdf = pd.DataFrame(row).T
        rowdf.columns = cols
        df = pd.concat([df, rowdf])

KeyboardInterrupt: 

In [ ]:
'mentions', 

In [262]:
# Create pd dataframes for features
cols = ['twitter_id', 'indeg', 'outdeg', 'clustercoeff', 'pagerank', 'indeg_A1', 
        'outdeg_A1', 'indeg_A2', 'outdeg_A2', 'label']

networks = ['likes', 'follows']

for net in networks:
    invalid = []

    df = pd.DataFrame(columns=cols)
    dirr = '../' + net + '/'
    for label in ['bot', 'human']:
        path = dirr + label + '/'
        for file in os.listdir(dirr + label + '/'):
            if os.path.isdir(file):
                continue
            full_path = path + file

            df, d[net] = extract_feats(full_path, df, invalid)

    df.to_pickle('feature_data/' + net + '.pkl')

No Network: ../likes/bot/790017240733278208
No Network: ../likes/bot/875984489138200577
No Network: ../likes/bot/1524730152
No Network: ../likes/bot/490645569
No Network: ../likes/bot/1416121214
No Network: ../likes/bot/2385417602
No Network: ../likes/bot/1421859620
No Network: ../likes/bot/4808016715
No Network: ../likes/bot/300415996
No Network: ../likes/human/2266976480
No Network: ../likes/human/757821654571425792
No Network: ../likes/human/1914493741
No Network: ../likes/human/804028724987494402
No Network: ../likes/human/938469204021469184
No Network: ../likes/human/3367811173
No Network: ../likes/human/837414074132606978
No Network: ../likes/human/2922179132
No Network: ../likes/human/750482565056765952
No Network: ../likes/human/1038380123710545920
No Network: ../likes/human/2257456093
No Network: ../likes/human/1054684530592878592
No Network: ../likes/human/957638029639913473
No Network: ../follows/bot/edgelist_2_edges_from_start_1421859620.pkl
No Network: ../follows/bot/edgel

In [265]:
pd.read_pickle('feature_data/likes.pkl').shape[0]

63

In [267]:
pd.read_pickle('feature_data/follows.pkl').shape[0]

76

In [268]:
pd.read_pickle('feature_data/mentions.pkl').shape[0]

451

In [248]:
full_path = '../likes/bot/790017240733278208'

In [249]:
uid = extract_uid(full_path)

# Another check if valid
is_valid(full_path, uid)

Error - Main twitter id not in edgelist: 2184009647 


False

In [251]:
edgelist = pd.read_pickle(full_path)
edgelist.originating_id = edgelist.originating_id.astype(str)
edgelist.receiving_id = edgelist.receiving_id.astype(str)
t = edgelist.originating_id.append(edgelist.receiving_id).unique()


In [253]:
edgelist

,originating_id,receiving_id,originating_following_count,originating_follower_count
